In [5]:
from langame_client import LangameClient
from pprint import pprint
from firebase_admin import firestore

In [6]:
langame_client = LangameClient()

In [29]:
# TODO: use protobuf
p = {}
p["type"] = "TopicGeneralist"
p["template"] = """This is a conversation starter that leads to extremely profound conversations between people about "[TOPIC]".
Starter: What do you think of the theory that animals have emotions?
Starter: What is the role of philosophy in human society?
Starter: How does one recognize truth and knowledge? What are the ways to acquire it?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 49, 26, 906587, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c13177610>)


In [33]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("transhumanism", "🤔🧠💭📚📖📐🧬🤖🦾🦿"),
)

calling openai with {'id': '4IpLQkq4VwnV4M3HheIg', 'prompt': 'It is a question that leads to extremely profound conversations between people about "transhumanism":\n1.', 'parameters': {'temperature': 0.7, 'topP': 1, 'stop': ['\\n', '2.'], 'presencePenalty': 0.1, 'model': 'davinci', 'frequencyPenalty': 0.1, 'maxTokens': 150}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 50, 4, 77231, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c88307730>)

In [39]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("politic", "🤔💭📜🏛👨👩⚖🗽"),
)

calling openai with {'id': '4IpLQkq4VwnV4M3HheIg', 'prompt': 'It is a question that leads to extremely profound conversations between people about "politic":\n1.', 'parameters': {'model': 'davinci', 'topP': 1, 'stop': ['\\n', '2.'], 'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'maxTokens': 150, 'temperature': 0.7}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 50, 58, 292063, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c13177250>)

In [36]:
langame_client.prompt_to_meme(topic_with_emoji = 
    ("mind", "🤔🧠💭📚📖"),
)

calling openai with {'id': '4IpLQkq4VwnV4M3HheIg', 'prompt': 'It is a question that leads to extremely profound conversations between people about "mind":\n1.', 'parameters': {'maxTokens': 150, 'temperature': 0.7, 'model': 'davinci', 'stop': ['\\n', '2.'], 'frequencyPenalty': 0.1, 'presencePenalty': 0.1, 'topP': 1}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 50, 29, 41695, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c13e6d3a0>)

In [10]:
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", ["ice breaker"])\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .limit(10)\
    .stream():
    pprint(t.reference.parent.parent.get().to_dict())

{'content': '"If money is the root of all evil, what is the root of money?"',
 'createdAt': DatetimeWithNanoseconds(2021, 7, 7, 20, 20, 53, 806000, tzinfo=<UTC>),
 'promptId': '4IpLQkq4VwnV4M3HheIg',
 'translated': {'de': '"Wenn Geld die Wurzel allen Übels ist, was ist dann die '
                      'Wurzel des Geldes?"',
                'es': '"Si el dinero es la raíz de todos los males, ¿cuál es '
                      'la raíz del dinero?"',
                'fr': "« Si l'argent est la racine de tous les maux, quelle "
                      "est la racine de l'argent ?"}}
{'content': '"Are we talking about a "state" or a "government"?"',
 'createdAt': DatetimeWithNanoseconds(2021, 7, 7, 20, 19, 57, 328000, tzinfo=<UTC>),
 'promptId': '4IpLQkq4VwnV4M3HheIg',
 'translated': {'de': '"Reden wir von einem "Staat" oder einer "Regierung"?"',
                'es': '"¿Estamos hablando de un" estado "o un" gobierno "?"',
                'fr': "« Parlons-nous d'un « État » ou d'un « gouvernem

In [16]:
# TODO: use protobuf
p = {}
p["type"] = "IceBreakerGeneralist"
p["template"] = """This is conversation starters that leads to extremely personal discussions between people.
Starter: What is an interesting fact that few people know about you?
Starter: What is something that most people take for granted, but has a deeper meaning to you?
Starter:"""
t = {
    "createdAt": firestore.SERVER_TIMESTAMP,
    "engine": {
        "parameters": {
            "model": "davinci",
            "temperature": 0.7,
            "maxTokens": 150,
            "topP": 1,
            "frequencyPenalty": 0.1,
            "presencePenalty": 0.1,
            "stop": ["\\n", "Starter:"],
        }
    }
}
p_col = langame_client._firestore_client.collection("prompts")
new_prompt_ref = p_col.add(p)
pprint(p_col.document(new_prompt_ref[1].id).collection("tags").add(t))

(DatetimeWithNanoseconds(2021, 7, 16, 15, 34, 18, 122873, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference object at 0x7f4c131a0fd0>)


In [28]:
langame_client.prompt_to_meme_ice_breaker()

calling openai with {'id': 'oCccJ82pVND4znQx3zM5', 'prompt': 'This is conversation starters that leads to extremely personal discussions between people.\nStarter: What is an interesting fact that few people know about you?\nStarter: What is something that most people take for granted, but has a deeper meaning to you?\nStarter:', 'parameters': {'temperature': 0.7, 'maxTokens': 150, 'frequencyPenalty': 0.1, 'model': 'davinci', 'presencePenalty': 0.1, 'stop': ['\\n', 'Starter:'], 'topP': 1}}


(DatetimeWithNanoseconds(2021, 7, 16, 15, 35, 59, 312111, tzinfo=datetime.timezone.utc),
 <google.cloud.firestore_v1.document.DocumentReference at 0x7f4c131a0dc0>)

In [6]:
for e in langame_client._firestore_client.collection("prompts")\
        .limit(5)\
        .stream():
    pprint(e.to_dict())

{'template': 'It is a question that leads to extremely profound conversations '
             'between people about "[TOPIC]":\n'
             '1.',
 'type': 'TopicGeneralist'}
{'template': 'Provide me a conversation starter that brings extremely deeply '
             'personal conversations between people, thus breaking the ice.\n'
             'Starter:',
 'type': 'IceBreakerGeneralist'}
{'template': 'This is a question that leads to extremely profound '
             'conversations between people and allow them to break the ice:\n'
             '1.',
 'type': 'IceBreakerGeneralist'}


In [23]:

completions = []
topics = ["philosophy", "physic", "mathematic", "biology", "transhumanism", "artificial intelligence", "book"]
for t in langame_client._firestore_client.collection_group(u"tags")\
    .where(u"topic.content", u"in", topics)\
    .order_by(u"createdAt", direction=firestore.Query.DESCENDING)\
    .stream():
    completions.append({"topic": t.to_dict()["topic"]["content"], "meme": t.reference.parent.parent.get().to_dict()["content"]})
pprint(len(completions))


AttributeError: 'str' object has no attribute 'write'

In [29]:
#{"prompt": "<prompt text>", "completion": "<ideal generated text>"}
def get_prompt(topic):
    return f"""This is a conversation starter that leads to extremely profound conversations between people about "{topic}".
Starter:"""

dataset = [{"prompt": get_prompt(e["topic"]), "completion": f" {e['meme']}"} for e in completions]

In [30]:
import json
with open('output.jsonl', 'w') as outfile:
    for entry in dataset:
        json.dump(entry, outfile)
        outfile.write('\n')

In [18]:
!openai tools fine_tunes.prepare_data -f output.jsonl

Analyzing...

- Your file contains 56 prompt-completion pairs. In general, we recommend having at least a few hundred examples. We've found that performance tends to linearly increase for every doubling of the number of examples

ERROR in common_suffix validator: All prompts are identical: `This is a conversation starter that leads to extremely profound conversations between people about "philosophy".
Starter:`
Consider leaving the prompts blank if you want to do open-ended generation, otherwise ensure prompts are different

Aborting...

In [ ]:
import os
import openai
openai.api_key = OPENAI_KEY
openai.File.list()

In [ ]:
openai.File.create(
  file=open("output.jsonl"),
  purpose="fine_tuning_topic_generalist"
)

In [ ]:
pprint(openai.File.list())

In [ ]:
openai.Completion.create(
  engine="babbage",
  prompt="Once upon a time",
  max_tokens=5
)